<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=af4a707d35f58b8cb3d29f0a6187b14117bea4f8ce784190ed4006671cf386fb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-08 14:43:19
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: 50.02 K (0.36%)
Current PnL: -17.85 L (-12.01%)
CY Booked + Current PnL: -7.99 L (-5.38%)
-------------------
Total profit:  2.88 L
Total loss:  -20.74 L
-------------------
Total Booked + Current PnL: 18.99 L (15.6%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.03%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.57 L (56.04%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.89%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-27.87,73.0,H-MC,8.01,144312.0,29376.0,7216.0,1.03,25.56,5.00,31.83,81.0,4.07,1.09,45.51,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-14.19,57.0,H-LC,10.47,171347.0,15170.0,10915.0,-0.21,9.71,6.37,16.71,22.0,1.39,1.29,28.85,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,104.70,62.0,M-SC,6.19,88163.0,-12614.0,12704.0,-0.98,-12.52,14.41,0.09,245.0,-0.99,0.66,15.83,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,51.0,M-LC,4.54,215312.0,13832.0,13392.0,0.73,6.87,6.22,13.51,60.0,1.03,1.62,20.33,XY25,NTT,BANKS
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.03,191835.0,40922.0,16383.0,2.20,27.12,8.54,37.97,93.0,2.50,1.44,59.28,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HAVELLS,2069.16,-0.09,61.0,H-MC,1.89,247588.0,-143.0,75192.0,0.29,-0.06,30.37,30.30,92.0,-0.00,1.86,13.23,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.81,40.0,H-MC,4.13,136190.0,57.0,31065.0,-1.66,0.04,22.81,22.86,91.0,0.00,1.02,12.66,X40,ATH,INSURANCE
53,PGHH,17973.08,-31.21,50.0,H-MC,4.88,199200.0,-1620.0,70397.0,-1.59,-0.81,35.34,34.25,80.0,-0.02,1.50,4.35,X40,ATH,FMCG
88,WIPRO,420.00,-14.25,39.0,M-LC,6.89,150579.0,-366.0,109817.0,-0.33,-0.24,72.93,72.51,53.0,-0.00,1.13,5.77,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-3.23,31.0,H-LC,10.07,221040.0,-20884.0,84150.0,-0.78,-8.63,38.07,26.15,8.0,-0.25,1.66,4.45,X40,ATH,IT
41,INFY,2275.00,-21.75,37.0,H-LC,10.89,305208.0,-7680.0,179371.0,-0.81,-2.45,58.77,54.87,3.0,-0.04,2.30,4.03,X40,BTT,IT
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284435.0,-61521.0,185366.0,-0.73,-17.78,65.17,35.80,1.0,-0.33,2.14,0.76,X40,BTT,IT
85,VBL,671.64,-15.98,38.0,H-LC,6.90,302348.0,-13494.0,126170.0,0.90,-4.27,41.73,35.67,5.0,-0.11,2.27,8.99,X40N,ATH,FMCG
82,UNITDSPR,1644.00,-14.51,44.0,H-LC,8.42,171877.0,-9882.0,45135.0,-0.76,-5.44,26.26,19.39,86.0,-0.22,1.29,2.27,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-6.94,46.0,H-LC,1.73,230535.0,-15616.0,94819.0,-1.40,-6.34,41.13,32.18,16.0,-0.16,1.73,27.41,X200,ATH,IT
52,NESTLEIND,1377.00,-9.37,57.0,H-LC,2.73,278717.0,13291.0,43508.0,-1.44,5.01,15.61,21.40,11.0,0.31,2.10,12.46,XY25,NTT,FMCG
43,ITC,452.00,-37.92,49.0,H-LC,3.12,199879.0,-259.0,21147.0,0.34,-0.13,10.58,10.44,4.0,-0.01,1.50,5.43,X40,NTT,FMCG
19,CIPLA,1795.00,-20.47,48.0,H-LC,5.75,212336.0,7836.0,33570.0,-0.23,3.83,15.81,20.25,10.0,0.23,1.60,12.90,X40N,BTT,PHARMA
85,VBL,671.64,-15.98,38.0,H-LC,6.90,302348.0,-13494.0,126170.0,0.90,-4.27,41.73,35.67,5.0,-0.11,2.27,8.99,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.0,-37.63,64.0,L-SC,38.64,92851.0,-20698.0,60752.0,8.53,-18.23,65.43,35.27,268.0,-0.34,0.70,127.26,XR,NTT,HOTELS
66,SHALBY,327.0,1096.17,79.0,M-SC,3.24,162765.0,-18378.0,63853.0,7.03,-10.15,39.23,25.10,235.0,-0.29,1.22,30.44,XY24,NTT,HEALTHCARE
55,RAJESHEXPO,518.0,1756.06,59.0,L-SC,8.28,50626.0,-86551.0,86646.0,5.66,-63.09,171.15,0.07,267.0,-1.00,0.38,25.91,OX40N,NTT,JEWELLERY
84,VALIANTORG,838.0,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS
75,TATAMOTORS,1065.0,-17.55,66.0,M-LC,2.56,261124.0,-15399.0,125470.0,4.00,-5.57,48.05,39.81,54.0,-0.12,1.96,25.15,XY24,NTT,AUTO


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,9.63,54.0,H-MC,3.32,189300.0,24630.0,43331.0,-2.60,14.96,22.89,41.28,89.0,0.57,1.42,24.13,X40N,ATH,AC
48,LAOPALA,464.00,110.10,45.0,H-SC,3.86,66770.0,-34076.0,59899.0,-1.74,-33.79,89.71,25.61,142.0,-0.57,0.50,23.79,AR,NTT,CERAMICS
56,RAJOOENG,143.33,-35.68,31.0,H-SC,11.09,97460.0,-5040.0,45874.0,-1.73,-4.92,47.07,39.83,114.0,-0.11,0.73,0.00,AR,ATH,MISC
36,ICICIGI,2260.25,-20.81,40.0,H-MC,4.13,136190.0,57.0,31065.0,-1.66,0.04,22.81,22.86,91.0,0.00,1.02,12.66,X40,ATH,INSURANCE
63,SATIN,274.00,-27.73,54.0,H-SC,8.13,142902.0,-53072.0,125897.0,-1.59,-27.08,88.10,37.16,126.0,-0.42,1.08,8.64,XY24,NTT,FINANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,104.70,62.0,M-SC,6.19,88163.0,-12614.0,12704.0,-0.98,-12.52,14.41,0.09,245.0,-0.99,0.66,15.83,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,89.03,53.0,H-SC,12.94,119070.0,-23409.0,42925.0,3.59,-16.43,36.05,13.70,163.0,-0.55,0.90,40.54,OX40N,NTT,BANKS
46,KANSAINER,340.0,-68.16,56.0,H-SC,2.87,219609.0,-50058.0,86394.0,0.47,-18.56,39.34,13.47,138.0,-0.58,1.65,11.08,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,115.96,51.0,M-SC,3.53,141509.0,-33050.0,33085.0,-0.37,-18.93,23.38,0.02,221.0,-1.00,1.06,23.09,OX40N,NTT,PAINTS
18,CERA,9475.0,-19.81,46.0,H-SC,2.53,114444.0,-29359.0,56100.0,0.27,-20.42,49.02,18.60,149.0,-0.52,0.86,26.29,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284435.0,-61521.0,185366.0,-0.73,-17.78,65.17,35.80,1.0,-0.33,2.14,0.76,X40,BTT,IT
41,INFY,2275.00,-21.75,37.0,H-LC,10.89,305208.0,-7680.0,179371.0,-0.81,-2.45,58.77,54.87,3.0,-0.04,2.30,4.03,X40,BTT,IT
43,ITC,452.00,-37.92,49.0,H-LC,3.12,199879.0,-259.0,21147.0,0.34,-0.13,10.58,10.44,4.0,-0.01,1.50,5.43,X40,NTT,FMCG
85,VBL,671.64,-15.98,38.0,H-LC,6.90,302348.0,-13494.0,126170.0,0.90,-4.27,41.73,35.67,5.0,-0.11,2.27,8.99,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,45.0,H-LC,12.92,246384.0,-15235.0,134452.0,0.30,-5.82,54.57,45.57,7.0,-0.11,1.85,4.84,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.03,191835.0,40922.0,16383.0,2.20,27.12,8.54,37.97,93.0,2.50,1.44,59.28,AR,ATH,AUTO
25,DIXON,18940.15,-27.87,73.0,H-MC,8.01,144312.0,29376.0,7216.0,1.03,25.56,5.00,31.83,81.0,4.07,1.09,45.51,X40N,ATH,IT
26,DMART,5201.00,-6.95,64.0,H-LC,16.07,232427.0,47946.0,22429.0,-1.04,25.99,9.65,38.14,38.0,2.14,1.75,39.35,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,44.35,64.0,M-MC,7.53,228182.0,3220.0,164428.0,2.51,1.43,72.06,74.53,192.0,0.02,1.72,32.48,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,9.63,54.0,H-MC,3.32,189300.0,24630.0,43331.0,-2.60,14.96,22.89,41.28,89.0,0.57,1.42,24.13,X40N,ATH,AC
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.03,191835.0,40922.0,16383.0,2.20,27.12,8.54,37.97,93.0,2.50,1.44,59.28,AR,ATH,AUTO
86,VOLTAS,1918.49,1.58,61.0,H-MC,3.69,213240.0,21498.0,74527.0,-0.78,11.21,34.95,50.08,99.0,0.29,1.60,18.77,XY25,ATH,AC
23,DABUR,735.00,-1.85,64.0,H-MC,4.83,213778.0,16750.0,72877.0,0.26,8.50,34.09,45.49,102.0,0.23,1.61,22.65,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.03,191835.0,40922.0,16383.0,2.20,27.12,8.54,37.97,93.0,2.50,1.44,59.28,AR,ATH,AUTO
51,NATIONALUM,247.44,-43.66,74.0,H-MC,1.22,112222.0,11718.0,18921.0,-0.19,11.66,16.86,30.49,79.0,0.62,0.84,47.84,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.40,62.0,M-SC,3.67,102405.0,10906.0,67843.0,0.62,11.92,66.25,86.07,223.0,0.16,0.77,48.71,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.09,49.0,H-SC,12.10,136427.0,13091.0,120670.0,0.74,10.61,88.45,108.45,119.0,0.11,1.03,35.34,AR,ATH,MISC
88,WIPRO,420.00,-14.25,39.0,M-LC,6.89,150579.0,-366.0,109817.0,-0.33,-0.24,72.93,72.51,53.0,-0.00,1.13,5.77,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.36,83568.0,-18642.0,112031.0,1.08,-18.24,134.06,91.37,208.0,-0.17,0.63,35.66,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-5.51,42.0,H-SC,15.36,88008.0,-12955.0,107484.0,-1.30,-12.83,122.13,93.63,148.0,-0.12,0.66,31.09,SR,ATH,CHEMICALS
50,MASFIN,398.61,-17.80,51.0,H-SC,12.88,93210.0,-4770.0,26369.0,-0.02,-4.87,28.29,22.05,152.0,-0.18,0.70,35.49,XR,ATH,FINANCE
21,COFFEEDAY,80.00,-37.63,64.0,L-SC,38.64,92851.0,-20698.0,60752.0,8.53,-18.23,65.43,35.27,268.0,-0.34,0.70,127.26,XR,NTT,HOTELS
84,VALIANTORG,838.00,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-37.63,64.0,L-SC,38.64,92851.0,-20698.0,60752.0,8.53,-18.23,65.43,35.27,268.0,-0.34,0.70,127.26,XR,NTT,HOTELS
7,ATULAUTO,844.00,3943.24,77.0,M-SC,8.62,126083.0,-21285.0,73103.0,-0.84,-14.44,57.98,35.16,236.0,-0.29,0.95,28.78,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.03,191835.0,40922.0,16383.0,2.20,27.12,8.54,37.97,93.0,2.50,1.44,59.28,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.36,83568.0,-18642.0,112031.0,1.08,-18.24,134.06,91.37,208.0,-0.17,0.63,35.66,XY25,NTT,FINANCE
66,SHALBY,327.00,1096.17,79.0,M-SC,3.24,162765.0,-18378.0,63853.0,7.03,-10.15,39.23,25.10,235.0,-0.29,1.22,30.44,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.68
1,25,43.52
2,50,73.76


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.23
LC    35.93
MC    25.79
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.12
X40      15.44
X40N     14.18
XY25     10.05
AR        9.51
XR        9.16
OX40N     7.48
MH        1.76
X200      1.73
X5K       1.43
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.61
H-MC    22.74
H-SC    22.65
M-SC    13.94
M-LC     7.24
M-MC     2.73
L-SC     1.64
L-LC     1.08
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-0.86,35.49
IT,13.29,-16.81,74.73
BANKS,7.44,-15.35,64.32
FINANCE,7.44,-22.08,67.09
PAINTS,6.04,-14.89,40.60
MISC,5.86,-17.99,85.62
ELECTRICAL,5.69,-8.79,68.75
AUTO,4.95,-2.58,47.22
AC,3.75,3.09,29.73


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2725818.0,22
AR,1214027.0,10
XR,1076926.0,13
X40,912932.0,10
X40N,547755.0,11
OX40N,463347.0,10
XY25,414839.0,7
SR,198017.0,2
X5K,127122.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2762776.0,27
H-LC,1448577.0,16
M-SC,1392148.0,17
H-MC,1179986.0,17
M-LC,414775.0,5
M-MC,314794.0,2
L-SC,243620.0,3
L-MC,59560.0,1
L-LC,40315.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,927042.0,6
M-SC,XY24,809877.0,7
H-SC,AR,663162.0,5
H-LC,X40,620217.0,5
H-SC,XR,516894.0,6
H-MC,XY24,457546.0,4
H-LC,AR,376272.0,2
M-MC,XY24,314794.0,2
H-LC,X40N,238219.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 58.0 seconds
